# Poznański Horyzont Danych Meetup - 10.12.2024
# DSPy Introduction

Tutorial inspired by [dspy/examples/agents/multi_agent.ipynb](https://github.com/stanfordnlp/dspy/blob/main/examples/agents/multi_agent.ipynb)

If you've ever felt intimidated by DSPy, don't worry—it might look complex at first glance, but it's actually quite approachable. This tutorial will walk you through the process of building a  project, providing a clear, step-by-step approach to understanding and implementing DSPy concepts.

## Useful links and concepts before starting to rock DSPy... 🚀

- I strongly recommend reading the [DSPy Cheatsheet](https://dspy.ai/cheatsheet/) it will help you with quick start;

- Do you want know more about DSPY optimizers? Read [this guide](https://dspy.ai/learn/optimization/optimizers/?h=optimizers)!

- You find tons of tutorials in [dpsy/examples](https://github.com/stanfordnlp/dspy/blob/main/examples) folder;

- Remember the DSPy "scientific method":
  - **Define your task**: what is the input? What is the expected output?
  - **Define your pipeline**: do we need ChainOfThoughts? Do we need any external tool, like a retriever?
  - **Explore a few examples**: explore both input and output. It might be worth to annotate few examples for debugging purposes.
  - **Define your data**: define training and validation sets;
  - **Define your metric**: define an objective metric you will use to evaluate your programs. You don't have it? You can ask an LLM to evaluate for you!
  - **Collect preliminary "zero-shot" evaluations**: try out your program without optimization to get baseline results;
  - **Compile with a DSPy optimizer**: start with FewShot optimizers and follow with more complex ones, like MIPRO, COPROv2...
  - **Iterate**: iterate the steps above, until you are satisfief with the results! 

In [61]:
import os
import re

import dspy
import pandas as pd
from dotenv import load_dotenv
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShot, LabeledFewShot, MIPROv2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# load your environment variables from .env file
load_dotenv()

# azure-openai model deployment
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

## Loading the Dataset

For this tutorial, we are going to use the [HotPotQA dataset](https://hotpotqa.github.io/). The task for the LLM is multi-hop question answering.

In [2]:
dataset = HotPotQA(train_seed=1, train_size=150, eval_seed=2023, dev_size=50, test_size=0)

In [3]:
dataset.train[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys=None)

### Specifying Inputs

It's crucial to inform DSPy which attributes serve as inputs for our model. We accomplish this using the `.with_inputs()` method.

In [4]:
train_set = [x.with_inputs('question') for x in dataset.train]
dev_set = [x.with_inputs('question') for x in dataset.dev]

In [5]:
train_set[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

## DSPy setup

DSPy is designed to be compatible with a variety of language models and their respective clients ([click here](https://dspy.ai/learn/programming/language_models) for a complete guide to all supported LLMs). For this tutorial, we will primarily utilize GPT-4 through the Azure client


In [6]:
llm = dspy.AzureOpenAI(
    api_base=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_VERSION,
    deployment_id=AZURE_OPENAI_DEPLOYMENT,
    api_key=AZURE_OPENAI_KEY,
    # Specify the models parameter
    temperature=0.7
)

retriever = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=llm, rm=retriever)

## Setting up Signatures and Modules


### Core DSPy Components

[dspy.Signature](https://dspy.ai/learn/programming/signatures/) and [dspy.Module](https://dspy.ai/learn/programming/modules/) are fundamental building blocks for DSPy programs:

- **Signature**: A declarative specification of the input/output behavior of a DSPy module.
- **Module**: A building block for programs that leverage Language Models (LMs).

### Types of DSPy Modules

DSPy offers various module types, each serving different purposes:

1. [dspy.Predict](https://dspy-docs.vercel.app/api/modules/Predict)
   - Basic predictor
   - Maintains the original signature
   - Handles key forms of learning (storing instructions, demonstrations, and LM updates)
   - Most similar to direct LM usage

2. [dspy.ChainOfThought](https://dspy-docs.vercel.app/api/modules/ChainOfThought)
   - Enhances the LM to think step-by-step before producing the final response
   - Modifies the signature to incorporate intermediate reasoning steps
     

3. [Additional Advanced Modules](https://dspy-docs.vercel.app/api/category/modules)
   - DSPy library offers a range of more specialized modules for complex tasks;
   - In this talk, we will explore the [ReAct](https://dspy.ai/deep-dive/modules/react/) module.

### Recommendation for starting

For those new to DSPy, it's advisable to start with `dspy.Predict`. Its simplicity makes it ideal for understanding the basics of DSPy operation. Once you've successfully implemented your program using `dspy.Predict`, you can explore more advanced modules like `dspy.ChainOfThought` to potentially enhance your model's performance.

For an overview of other prompting techniques beyond zero-shot learning, refer to the [Prompting Guide](https://www.promptingguide.ai/techniques). This resource covers various methods that can enhance your DSPy applications as you progress.

In [35]:
class GenerateAnswer(dspy.Signature):
    # [OPTIONAL]: Clarify something about the nature of the task expressed below as a docstring.
    """Answer questions with short factoid answers."""

    # Supply hints on the nature of an input field, expressed as a desc keyword argument for dspy.InputField.
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [48]:
class BaselineRAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer, rationale=prediction.rationale)

In [49]:
baseline = BaselineRAG()

baseline(train_set[0].question)

Prediction(
    context=['At My Window (album) | At My Window is an album released by Folk/country singer-songwriter Townes Van Zandt in 1987. This was Van Zandt\'s first studio album in the nine years that followed 1978\'s "Flyin\' Shoes", and his only studio album recorded in the 1980s. Although the songwriter had become less prolific, this release showed that the quality of his material remained high.', 'Little Window | Little Window is the debut album of American singer-songwriter Baby Dee. The album was released in 2002 on the Durtro label. It was produced, composed, and performed entirely by Dee.', 'Windows and Walls | Windows and Walls is the eighth album by American singer-songwriter Dan Fogelberg, released in 1984 (see 1984 in music). The first single, "The Language of Love", reached 13 on the U.S. "Billboard" Hot 100 chart. Although the follow-up, "Believe in Me", missed the Top 40 of the pop chart, peaking at No. 48, it became the singer\'s fourth No. 1 song on the "Billboar

## Metrics & Evaluation

In [50]:
# We have a positive match when:
# (generated answer and true answer match exactly) AND (the retrieved context does actually contain that answer)
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

In [51]:
evaluate_hotpot = Evaluate(
    devset=dev_set,
    metric=validate_context_and_answer,
    num_threads=8,
    display_progress=True,
    display_table=10,
)

In [52]:
evaluate_hotpot(baseline)

Average Metric: 18.00 / 50 (36.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 169.43it/s]

2024/12/09 13:04:34 INFO dspy.evaluate.evaluate: Average Metric: 18 / 50 (36.0%)


question  \
0         Are both Cangzhou and Qionghai in the Hebei province of China?   
1  Who conducts the draft in which Marc-Andre Fleury was drafted to t...   
2  The Wings entered a new era, following the retirement of which Can...   
3                     What river is near the Crichton Collegiate Church?   
4  In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...   
5  The Newark Airport Exchange is at the northern edge of an airport ...   
6  Where did an event take place resulting in a win during a domestic...   
7  Are both Chico Municipal Airport and William R. Fairchild Internat...   
8                         In which Maine county is Fort Pownall located?   
9  Which 90s rock band has more recently reformed, Gene or The Afghan...   

                              example_answer  \
0                                         no   
1                     National Hockey League   
2                              Steve Yzerman   
3                             the River Tyne   
4                      King Alfred the Great   
5  Port Authority of New York and New Jersey   
6                                 Bundesliga   
7                                         no   
8                        Waldo County, Maine   
9                           The Afghan Whigs   

                                                             gold_titles  \
0                                                   {Cangzhou, Qionghai}   
1         {2017 NHL Expansion Draft, 2017–18 Pittsburgh Penguins season}   
2                      {Steve Yzerman, 2006–07 Detroit Red Wings season}   
3                          {Crichton Collegiate Church, Crichton Castle}   
4                                {Æthelweard (son of Alfred), Ealhswith}   
5     {Newark Airport Interchange, Newark Liberty International Airport}   
6                     {2005–06 FC Bayern Munich season, Claudio Pizarro}   
7  {Chico Municipal Airport, William R. Fairchild International Airport}   
8                                {Fort Pownall, Stockton Springs, Maine}   
9                                        {The Afghan Whigs, Gene (band)}   

                                                                 context  \
0  ['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...   
1  ['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...   
2  ['Steve Yzerman | Stephen Gregory "Steve" Yzerman ( ; born May 9, ...   
3  ["Crichton Collegiate Church | Crichton Collegiate Church is situa...   
4  ["Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...   
5  ['Newark Airport Interchange | The Newark Airport Interchange is a...   
6  ['List of Peru international footballers | Peru took part in the i...   
7  ['William R. Fairchild International Airport | William R. Fairchil...   
8  ["Fort Pownall | Fort Pownall was a British fortification built du...   
9  ['The Afghan Whigs | The Afghan Whigs are an American rock band fr...   

                                 pred_answer  \
0                         No, only Cangzhou.   
1                                        NHL   
2                              Steve Yzerman   
3                                 River Tyne   
4                           Alfred the Great   
5  Port Authority of New York and New Jersey   
6                                       Peru   
7                                         No   
8                               Waldo County   
9                           The Afghan Whigs   

                                                               rationale  \
0  determine the location of both cities. We know that Cangzhou is in...   
1  identify the entity responsible for the draft in question. We know...   
2  identify the player mentioned in the context. We see that Steve Yz...   
3  identify the river associated with Crichton Collegiate Church. The...   
4  identify the king who was the father of Æthelweard. Ealhswith is k...   
5  identify the operator of Newark Libe

36.0

## And now... some DSPy magic ✨

### Using a ReAct module

The ReAct framework combines reasoning (thinking) and acting (taking actions) in LLMs.
It alternates between reasoning about the problem and performing actions, like retrieving information or interacting with an environment, to dynamically adapt as new information becomes available.

<center><img src="assets/react-schema.png" width="1000" align="center"><center>

In [53]:
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

In [54]:
question = train_set[0].question
result = agent(question=question)
result.trajectory

{'thought_0': 'At My Window was released by the American singer-songwriter Joni Mitchell.\n\nNext Thought: I need to confirm the artist for the song "At My Window" to ensure accuracy.',
 'tool_name_0': 'Search',
 'tool_args_0': '{"query": "At My Window American singer-songwriter"}',
 'observation_0': Prediction(
     passages=['At My Window (album) | At My Window is an album released by Folk/country singer-songwriter Townes Van Zandt in 1987. This was Van Zandt\'s first studio album in the nine years that followed 1978\'s "Flyin\' Shoes", and his only studio album recorded in the 1980s. Although the songwriter had become less prolific, this release showed that the quality of his material remained high.']
 ),
 'thought_1': 'It seems that the song "At My Window" is actually attributed to Townes Van Zandt, not Joni Mitchell. I need to update my answer accordingly.',
 'tool_name_1': 'finish',
 'tool_args_1': '{}',
 'observation_1': 'Completed.'}

In [55]:
config = dict(num_threads=8, display_progress=True, display_table=5)
evaluate = Evaluate(devset=dev_set, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)

Average Metric: 22.00 / 50 (44.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 32.58it/s]

2024/12/09 13:08:33 INFO dspy.evaluate.evaluate: Average Metric: 22 / 50 (44.0%)


question  \
0         Are both Cangzhou and Qionghai in the Hebei province of China?   
1  Who conducts the draft in which Marc-Andre Fleury was drafted to t...   
2  The Wings entered a new era, following the retirement of which Can...   
3                     What river is near the Crichton Collegiate Church?   
4  In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...   

           example_answer  \
0                      no   
1  National Hockey League   
2           Steve Yzerman   
3          the River Tyne   
4   King Alfred the Great   

                                                      gold_titles  \
0                                            {Cangzhou, Qionghai}   
1  {2017 NHL Expansion Draft, 2017–18 Pittsburgh Penguins season}   
2               {Steve Yzerman, 2006–07 Detroit Red Wings season}   
3                   {Crichton Collegiate Church, Crichton Castle}   
4                         {Æthelweard (son of Alfred), Ealhswith}   

                                                              trajectory  \
0  {'thought_0': 'Cangzhou is located in Hebei province, while Qiongh...   
1  {'thought_0': "The question asks about the draft process for Marc-...   
2  {'thought_0': "The Wings' new era likely began after a significant...   
3  {'thought_0': 'Question: What river is near the Crichton Collegiat...   
4  {'thought_0': "Question: In the 10th Century A.D. Ealhswith had a ...   

                                                               rationale  \
0  determine whether both cities are in the Hebei province. We start ...   
1  determine who conducted the draft in which Marc-Andre Fleury was d...   
2  produce the answer. We started by identifying the context of the q...   
3  determine the river near the Crichton Collegiate Church. I started...   
4  determine which English king Ealhswith was married to and subseque...   

                                                             pred_answer  \
0                  No, they are not both in the Hebei province of China.   
1  The NHL conducted the draft in which Marc-Andre Fleury was drafted...   
2                                                          Steve Yzerman   
3                                                             River Tyne   
4                                                  King Alfred the Great   

  answer_exact_match  
0                     
1                     
2          ✔️ [True]  
3          ✔️ [True]  
4          ✔️ [True]

44.0

### Using Optimizers - Automatic Few-Shot Learning

These optimizers extend the signature by automatically generating and including optimized examples within the prompt sent to the model, implementing few-shot learning.

#### LabeledFewShot

Simply constructs few-shot examples (demos) from provided labeled input and output data points.

In [66]:
optimizer = LabeledFewShot(k=3)
optimized_program = optimizer.compile(baseline, trainset=train_set)
optimized_program.save("compiled_programs/LabeledFewShot_program.json")

evaluate(optimized_program)

Average Metric: 28.00 / 50 (56.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 293.83it/s]

2024/12/09 13:27:02 INFO dspy.evaluate.evaluate: Average Metric: 28 / 50 (56.0%)


question  \
0         Are both Cangzhou and Qionghai in the Hebei province of China?   
1  Who conducts the draft in which Marc-Andre Fleury was drafted to t...   
2  The Wings entered a new era, following the retirement of which Can...   
3                     What river is near the Crichton Collegiate Church?   
4  In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...   

           example_answer  \
0                      no   
1  National Hockey League   
2           Steve Yzerman   
3          the River Tyne   
4   King Alfred the Great   

                                                      gold_titles  \
0                                            {Cangzhou, Qionghai}   
1  {2017 NHL Expansion Draft, 2017–18 Pittsburgh Penguins season}   
2               {Steve Yzerman, 2006–07 Detroit Red Wings season}   
3                   {Crichton Collegiate Church, Crichton Castle}   
4                         {Æthelweard (son of Alfred), Ealhswith}   

                                                                 context  \
0  ['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...   
1  ['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...   
2  ['Steve Yzerman | Stephen Gregory "Steve" Yzerman ( ; born May 9, ...   
3  ["Crichton Collegiate Church | Crichton Collegiate Church is situa...   
4  ["Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...   

              pred_answer  \
0                      No   
1  National Hockey League   
2           Steve Yzerman   
3              River Tyne   
4        Alfred the Great   

                                                               rationale  \
0  determine the locations of Cangzhou and Qionghai. Cangzhou is conf...   
1  identify the entity responsible for conducting the draft. The cont...   
2  identify the retired professional ice hockey player mentioned. The...   
3  identify the river associated with the Crichton Collegiate Church....   
4  identify the king who was the father of Æthelweard. From the conte...   

  answer_exact_match  
0          ✔️ [True]  
1          ✔️ [True]  
2          ✔️ [True]  
3          ✔️ [True]  
4

56.0

### Using Optimizers - Automatic Instruction Optimization

These optimizers produce optimal instructions for the prompt.

#### MIPROv2

Generates instructions and few-shot examples in each step. The instruction generation is data-aware and demonstration-aware. Uses Bayesian Optimization to effectively search over the space of generation instructions/demonstrations across your modules.

<center><img src="assets/mipro-schema.png" width="400"></center>

In [73]:
optimizer = MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",                      # Can choose between light, medium, and heavy optimization runs
    verbose=True
)
optimized_program = optimizer.compile(
    baseline,
    trainset=train_set,
    max_bootstrapped_demos=0,           # setting this to 0 to optimize only the instruction
    max_labeled_demos=0,                # setting this to 0 to optimize only the instructions
    requires_permission_to_run=True
)

optimized_program.save("compiled_programs/MIPROv2_program.json")

evaluate(optimized_program)

2024/12/09 13:50:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 7
valset size: 100



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program-aware proposer = 19 prompt model calls
- Program Evaluation: 25 examples in minibatch * 7 batches + 100 examples in val set * 1 full evals = 275 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectati

2024/12/09 13:50:22 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/12/09 13:50:22 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2024/12/09 13:50:22 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Bootstrapping set 1/7
Bootstrapping set 2/7



17%|███████████████████████████████▋                                                                                                                                                              | 5/30 [00:00<00:00, 304.17it/s]

...

2024/12/09 13:50:24 INFO dspy.evaluate.evaluate: Average Metric: 41 / 100 (41.0%)
2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 41.0
2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [39.0, 41.0]
2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 41.0
2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: 

2024/12/09 13:50:24 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 41.0!


Average Metric: 21.00 / 50 (42.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 279.67it/s]

2024/12/09 13:50:24 INFO dspy.evaluate.evaluate: Average Metric: 21 / 50 (42.0%)


question  \
0         Are both Cangzhou and Qionghai in the Hebei province of China?   
1  Who conducts the draft in which Marc-Andre Fleury was drafted to t...   
2  The Wings entered a new era, following the retirement of which Can...   
3                     What river is near the Crichton Collegiate Church?   
4  In the 10th Century A.D. Ealhswith had a son called Æthelweard by ...   

           example_answer  \
0                      no   
1  National Hockey League   
2           Steve Yzerman   
3          the River Tyne   
4   King Alfred the Great   

                                                      gold_titles  \
0                                            {Cangzhou, Qionghai}   
1  {2017 NHL Expansion Draft, 2017–18 Pittsburgh Penguins season}   
2               {Steve Yzerman, 2006–07 Detroit Red Wings season}   
3                   {Crichton Collegiate Church, Crichton Castle}   
4                         {Æthelweard (son of Alfred), Ealhswith}   

                                                                 context  \
0  ['Cangzhou | Cangzhou () is a prefecture-level city in eastern Heb...   
1  ['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Peng...   
2  ['Steve Yzerman | Stephen Gregory "Steve" Yzerman ( ; born May 9, ...   
3  ["Crichton Collegiate Church | Crichton Collegiate Church is situa...   
4  ["Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-cent...   

              pred_answer  \
0   No, only Cangzhou is.   
1  National Hockey League   
2           Steve Yzerman   
3              River Tyne   
4        Alfred the Great   

                                                               rationale  \
0  determine the location of both cities. Cangzhou is explicitly ment...   
1  identify the organization responsible for the expansion draft. The...   
2  identify the player being referred to. The context states that the...   
3  identify the relevant geographical features. The context mentions ...   
4  identify the correct English king associated with Ealhswith and he...   

  answer_exact_match  
0                     
1          ✔️ [True]  
2          ✔️ [True]  
3          ✔️ [True]  
4

42.0

In [72]:
llm.inspect_history(n=1)




You are a knowledgeable trivia expert who specializes in providing concise and accurate answers to fact-based questions. Use the provided context to identify the relevant information and respond with a brief answer, ensuring clarity and precision.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «Sasha Alexander | Suzana S. Drobnjaković Ponti (born May 17, 1973), known by her stage name Sasha Alexander, is a Serbian-American actress. She played Gretchen Witter on "Dawson's Creek" and has acted in films including "Yes Man" (2008) and "He's Just Not That Into You" (2009). Alexander played Caitlin Todd for the first two seasons of "NCIS". From July 2010 through September 2016, Alexander starred as Maura Isles in the TNT series "Rizzoli & Isles".»
[2] «The Hour of the Star | The Hour of the Star ("A hora da e

'\n\n\nYou are a knowledgeable trivia expert who specializes in providing concise and accurate answers to fact-based questions. Use the provided context to identify the relevant information and respond with a brief answer, ensuring clarity and precision.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: often between 1 and 5 words\n\n---\n\nContext:\n[1] «Sasha Alexander | Suzana S. Drobnjaković Ponti (born May 17, 1973), known by her stage name Sasha Alexander, is a Serbian-American actress. She played Gretchen Witter on "Dawson\'s Creek" and has acted in films including "Yes Man" (2008) and "He\'s Just Not That Into You" (2009). Alexander played Caitlin Todd for the first two seasons of "NCIS". From July 2010 through September 2016, Alexander starred as Maura Isles in the TNT series "Rizzoli & Isles".»\n[2] «The Hour of the Star | The Hour 